# Tutoriel d'usage de l'API

In [1]:
import urllib.parse

domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

## Creating an avatar

The admin of the competition can create users. A user can then create as many avatars as they wish.

In [2]:
import requests

user_id = '12e1e6fc-456b-4738-ba59-b90c3386f583'
name = 'ema-first-avatar'
r = requests.post(path(f'avatars/{user_id}/{name}'))
r

<Response [409]>

In [3]:
# name = 'max-second-avatar'
# r = requests.post(path(f"avatars/{user_id}/{name}"))
# r

<Response [200]>

In [3]:
r = requests.get(path(f"avatars/{user_id}"))
for avatar in r.json():
    print(avatar['id'], avatar['name'])

6217 ema-first-avatar


## Making pricing requests

In [4]:
params = {
    "avatar_name": "ema-first-avatar",
    "language": "hungarian",
    "city": "amsterdam",
    "date": 30,
    "mobile": 0,
}
r1 = requests.get(path(f"pricing/{user_id}"), params=params)
r1.json()

{'prices': [{'hotel_id': 585, 'price': 75, 'stock': 29},
  {'hotel_id': 917, 'price': 257, 'stock': 118},
  {'hotel_id': 410, 'price': 115, 'stock': 38},
  {'hotel_id': 45, 'price': 152, 'stock': 54},
  {'hotel_id': 510, 'price': 259, 'stock': 119},
  {'hotel_id': 806, 'price': 259, 'stock': 102},
  {'hotel_id': 234, 'price': 144, 'stock': 52},
  {'hotel_id': 627, 'price': 106, 'stock': 41},
  {'hotel_id': 676, 'price': 69, 'stock': 20},
  {'hotel_id': 429, 'price': 241, 'stock': 110},
  {'hotel_id': 507, 'price': 108, 'stock': 36},
  {'hotel_id': 82, 'price': 109, 'stock': 44},
  {'hotel_id': 211, 'price': 149, 'stock': 60},
  {'hotel_id': 679, 'price': 152, 'stock': 52},
  {'hotel_id': 475, 'price': 97, 'stock': 49},
  {'hotel_id': 86, 'price': 107, 'stock': 78},
  {'hotel_id': 630, 'price': 145, 'stock': 66},
  {'hotel_id': 103, 'price': 182, 'stock': 80},
  {'hotel_id': 223, 'price': 149, 'stock': 51},
  {'hotel_id': 264, 'price': 77, 'stock': 21},
  {'hotel_id': 442, 'price': 162,

In [6]:
# params = {
#     "avatar_name": "max-first-avatar",
#     "language": "hungarian",
#     "city": "copenhagen",
#     "date": 3,
#     "mobile": 0,
# }
# r = requests.get(path(f"pricing/{user_id}"), params=params)
# r.status_code, r.json()['detail']

(422, 'A pricing request for this avatar already exists for a sooner date')

In [5]:
params = {
    "avatar_name": "ema-first-avatar",
    "language": "hungarian",
    "city": "amsterdam",
    "date": 30,
    "mobile": 1,
}
r2 = requests.get(path(f"pricing/{user_id}"), params=params)
r2.json()

{'prices': [{'hotel_id': 969, 'price': 78, 'stock': 31},
  {'hotel_id': 158, 'price': 142, 'stock': 62},
  {'hotel_id': 585, 'price': 76, 'stock': 29},
  {'hotel_id': 34, 'price': 75, 'stock': 23},
  {'hotel_id': 253, 'price': 250, 'stock': 105},
  {'hotel_id': 627, 'price': 108, 'stock': 41},
  {'hotel_id': 891, 'price': 106, 'stock': 22},
  {'hotel_id': 917, 'price': 261, 'stock': 118},
  {'hotel_id': 762, 'price': 159, 'stock': 50},
  {'hotel_id': 196, 'price': 112, 'stock': 19},
  {'hotel_id': 779, 'price': 182, 'stock': 81},
  {'hotel_id': 269, 'price': 152, 'stock': 60},
  {'hotel_id': 224, 'price': 146, 'stock': 57},
  {'hotel_id': 821, 'price': 106, 'stock': 45},
  {'hotel_id': 724, 'price': 260, 'stock': 93},
  {'hotel_id': 172, 'price': 119, 'stock': 38},
  {'hotel_id': 10, 'price': 155, 'stock': 51},
  {'hotel_id': 715, 'price': 146, 'stock': 50},
  {'hotel_id': 892, 'price': 184, 'stock': 69},
  {'hotel_id': 111, 'price': 116, 'stock': 40},
  {'hotel_id': 807, 'price': 269,

## Build a training set

In [6]:
import pandas as pd

pricing_requests = []

requests = [r1,r2]
for r in requests:
    pricing_requests.append(
        pd.DataFrame(r.json()['prices']).assign(**r.json()['request'])
    )

pricing_requests = pd.concat(pricing_requests)
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,585,75,29,amsterdam,30,hungarian,0,6217
1,917,257,118,amsterdam,30,hungarian,0,6217
2,410,115,38,amsterdam,30,hungarian,0,6217
3,45,152,54,amsterdam,30,hungarian,0,6217
4,510,259,119,amsterdam,30,hungarian,0,6217


In [14]:
# print(pricing_requests[ pricing_requests['mobile'] == 1 ])

     hotel_id  price  stock       city  date   language  mobile  avatar_id
0         809    238    109  amsterdam    30  hungarian       1       6217
1         429    233    110  amsterdam    30  hungarian       1       6217
2         375    101     42  amsterdam    30  hungarian       1       6217
3         548    223    126  amsterdam    30  hungarian       1       6217
4         777    137     60  amsterdam    30  hungarian       1       6217
..        ...    ...    ...        ...   ...        ...     ...        ...
128       529    223    115  amsterdam    30  hungarian       1       6217
129       658    154     56  amsterdam    30  hungarian       1       6217
130       103    176     80  amsterdam    30  hungarian       1       6217
131       450    167     86  amsterdam    30  hungarian       1       6217
132       214    173     72  amsterdam    30  hungarian       1       6217

[133 rows x 8 columns]


In [7]:
len(pricing_requests)

266

In [10]:
data=pd.read_csv('./Datatrain/test_requete.csv')

In [19]:
data=data.drop(['Unnamed: 0'],axis=1)

In [22]:
pricing_requests=pd.concat([data,pricing_requests])

In [23]:
pricing_requests

410

In [29]:
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,556,175,60,copenhagen,30,hungarian,0,6217,Boss Western,J.Halliday Inn,1,0,0
1,664,284,111,copenhagen,30,hungarian,0,6217,Accar Hotels,Safitel,0,1,0
2,96,124,37,copenhagen,30,hungarian,0,6217,Boss Western,Boss Western,0,0,0
3,865,277,107,copenhagen,30,hungarian,0,6217,Chillton Worldwide,Chill Garden Inn,1,1,0
4,58,203,65,copenhagen,30,hungarian,0,6217,Yin Yang,Royal Lotus,1,0,2


In [30]:
pricing_requests.to_csv('./Datatrain/test_requete.csv')

In [11]:
# sample_submission = pd.read_csv('../input/defi-ia-2023/sample_submission.csv')
# sample_submission.head()

,index,price
0,0,191.0
1,1,191.0
2,2,191.0
3,3,191.0
4,4,191.0


In [12]:
# sample_submission.to_csv('submission.csv', index=False)